In [2]:
from torchrl.data.replay_buffers import (
    TensorDictReplayBuffer,
    LazyTensorStorage,
    PrioritizedSampler,
)
from tensordict import TensorDict

import torch
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import gymnasium as gym
import numpy as np

from rlarcworld.arc_dataset import ArcDataset, ArcSampleTransformer
from rlarcworld.enviroments.arc_batch_grid_env import ArcBatchGridEnv
from rlarcworld.enviroments.wrappers.rewards import PixelAwareRewardWrapper
from rlarcworld.agent.actor import ArcActorNetwork
from rlarcworld.agent.critic import ArcCriticNetwork

from rlarcworld.algorithms.d4pg import D4PG

2025-02-13 08:18:31 - arc_batch_grid_env.py - INFO - Registering gymnasium environment


In [ ]:

env = ArcBatchGridEnv(
    size=30,
    color_values=11,
    n_steps=1,
    gamma=1.0,
)

env = PixelAwareRewardWrapper(env, n_steps=1, gamma=1.0)
isinstance(env, PixelAwareRewardWrapper), isinstance(env.env, gym.Env)

(True, True)

In [5]:
dataset = ArcDataset(
            arc_dataset_dir="./dataset/training",
            keep_in_memory=True,
            transform=ArcSampleTransformer(
                (30, 30), examples_stack_dim=10
            ),
        )
train_samples = DataLoader(dataset=dataset, batch_size=10)

In [6]:
train_samples.batch_size

10

In [8]:
isinstance(train_samples, DataLoader), isinstance(train_samples.dataset, ArcDataset)

(True, True)

In [5]:
x = {}

In [6]:
x["pixel_wise"]=x.setdefault("pixel_wise", 0) + 1

In [7]:
x

{'pixel_wise': 1}

In [3]:
td = TensorDict({"a": torch.zeros(10), "b": torch.ones(10)}, [])
td.shape[0]

IndexError: tuple index out of range

In [5]:
import torch
with torch.enable_grad():
    torch.zeros(10)

In [ ]:
print(torch.nn.functional.gumbel_softmax(torch.randn(10), tau=1, hard=False).unsqueeze(-1))
print(torch.softmax(torch.randn(10), dim=-1).unsqueeze(-1))

tensor([[0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]])
tensor([[0.2633],
        [0.0215],
        [0.0053],
        [0.1604],
        [0.0951],
        [0.0764],
        [0.0274],
        [0.1028],
        [0.2293],
        [0.0185]])
